### 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

### 02. Importing data frame

In [2]:
ords_prods_cust = pd.read_pickle(r'C:\Users\veren\Python Data\26-10-2021 Instacart Basket Analysis\02 Data\Prepared Data\ords_prods_cust_region.pkl')

In [3]:
ords_prods_cust.shape

(30356128, 31)

In [4]:
ords_prods_cust.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,frequency,frequency_flag,gender,state,age,date_joined,n_dependants,family_status,income,region
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
2,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
3,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest
4,3265,Mini Seedless Watermelon Pack,32,4,1.7,960220,138,17,5,14,...,8.0,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest


In [5]:
ords_prods_cust.columns

Index(['product_id', 'product_name', 'aisle_id', 'department_id', 'prices',
       'order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'add_to_cart_order', 'reordered',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'spending',
       'spending_flag', 'frequency', 'frequency_flag', 'gender', 'state',
       'age', 'date_joined', 'n_dependants', 'family_status', 'income',
       'region'],
      dtype='object')

### 03. Comparing regional spending

In [6]:
# Crossing the region variable with the spending_flag
crosstab = pd.crosstab(ords_prods_cust['region'], ords_prods_cust['spending_flag'], dropna = False)

In [7]:
crosstab

spending_flag,High Spender,Low Spender
region,,
Midwest,148545,6965077
Northeast,103091,5258410
South,197563,9911312
West,150117,7622013


The South region has the most high spenders, followed by the West, the Midwest, and the Northeast. But, the South also has the most low spenders, followed by the West, the Midwest, and the Northeast. Overall, these numbers seem to be correlated to the population counts in these regions because the most populated region is the South, followed by the West, the Midwest, and the Northeast.

In [8]:
crosstab.to_clipboard

<bound method NDFrame.to_clipboard of spending_flag  High Spender  Low Spender
region                                  
Midwest              148545      6965077
Northeast            103091      5258410
South                197563      9911312
West                 150117      7622013>

### 04. Creating an exclusion flag for low-activity customers

In [9]:
# Creating a flag for low-activity customers (who have less than 5 orders)
ords_prods_cust.loc[ords_prods_cust['max_order'] < 5, 'exclusion_flag'] = 'Low-activity Customer'

In [10]:
# Creating a flag for customers with a higher activity (who have 5 or more orders). I call them non_low_activity customer.
ords_prods_cust.loc[ords_prods_cust['max_order'] >= 5, 'exclusion_flag'] = 'Non-low-activity Customer'

In [11]:
# Checking the creation of the flag with the head()-function
ords_prods_cust[['max_order', 'exclusion_flag']].head(10)

,max_order,exclusion_flag
0,32,Non-low-activity Customer
1,32,Non-low-activity Customer
2,32,Non-low-activity Customer
3,32,Non-low-activity Customer
4,32,Non-low-activity Customer
5,32,Non-low-activity Customer
6,32,Non-low-activity Customer
7,32,Non-low-activity Customer
8,32,Non-low-activity Customer
9,32,Non-low-activity Customer


In [12]:
ords_prods_cust['exclusion_flag'].value_counts(dropna = False)

Non-low-activity Customer    29329778
Low-activity Customer         1026350
Name: exclusion_flag, dtype: int64

### 05. Creating a dataframe for low-activity customers and non-low-activity customers

In [13]:
# Creating a new dataframe only showing data for low-activity customers
df_low_cust = ords_prods_cust[ords_prods_cust['exclusion_flag'] == 'Low-activity Customer']

In [14]:
df_low_cust.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,frequency_flag,gender,state,age,date_joined,n_dependants,family_status,income,region,exclusion_flag
295,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308,Midwest,Low-activity Customer
296,3260,Chips Ahoy!/Nutter Butter/Oreo Cookies,61,19,1.7,705212,764,2,2,10,...,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308,Midwest,Low-activity Customer
297,3260,Chips Ahoy!/Nutter Butter/Oreo Cookies,61,19,1.7,1813452,764,3,4,17,...,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308,Midwest,Low-activity Customer
298,4047,Milano,61,19,4.4,705212,764,2,2,10,...,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308,Midwest,Low-activity Customer
299,4377,Bag of Oranges,24,4,10.9,1813452,764,3,4,17,...,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308,Midwest,Low-activity Customer


In [15]:
df_low_cust.shape

(1026350, 32)

In [16]:
# Creating a new dataframe only showing data for non-low-activity customers
df_normal_cust = ords_prods_cust[ords_prods_cust['exclusion_flag'] == 'Non-low-activity Customer']

In [17]:
df_normal_cust.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,order_dow,order_hour_of_day,...,frequency_flag,gender,state,age,date_joined,n_dependants,family_status,income,region,exclusion_flag
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,Non-low-activity Customer
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,Non-low-activity Customer
2,907,Premium Sliced Bacon,106,12,20.0,2254091,138,10,5,14,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,Non-low-activity Customer
3,1000,Apricots,18,10,12.9,505689,138,9,6,12,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,Non-low-activity Customer
4,3265,Mini Seedless Watermelon Pack,32,4,1.7,960220,138,17,5,14,...,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620,Midwest,Non-low-activity Customer


In [18]:
df_normal_cust.shape

(29329778, 32)

In [19]:
# Checking if df_normal_cust and df_low_cust together have the same number of rows than ords_prods_cust
29329778 + 1026350

30356128

Success!

### 06. Exporting dataframes

In [20]:
path = r'C:\Users\veren\Python Data\26-10-2021 Instacart Basket Analysis'

In [21]:
path

'C:\\Users\\veren\\Python Data\\26-10-2021 Instacart Basket Analysis'

In [22]:
# Exporting df_normal_cust
df_normal_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'normal_customers.pkl'))

In [24]:
# Exporting ords_prods_cust
ords_prods_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'final_ords_prods_cust_whole.pkl'))